Script to output the necessary fields for the CALP constraint for CMIP6.  This includes...

- Interannual correlation between DJF Nino3.4 and California precipitation for 1948-2014 of historical (after detrending)
- Interannual correlation between DJF Nino3.4 and California precipitation for 2006-2099 of historical and SSP5-8.5 concatenated (after detrending)
- DJF trend in precipitation from 2006-2099 in historical and SSP5-8.5 concatenated.

Output is put to ecpaper2020/DATASORT/CALP/DATA

In [14]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import xesmf as xe
from numpy import nan
import sys
import warnings
from scipy import signal
from glob import glob
import math

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import shapefile_utils as shp
from ecpaper_utils import averaging_utils as avg
from ecpaper_utils import linfit_utils as linfit

from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO

importlib.reload(read)
importlib.reload(cal)
importlib.reload(shp)
importlib.reload(avg)

warnings.filterwarnings('ignore')

Set paths for CMIP6 models (historical and SSP5-8.5)

In [15]:
histpath="/project/cmip6/historical/Amon/"
rcp85path="/project/cmip6/ssp585/Amon/"
pathout="/project/cas/islas/python/ecpaper2020/DATASORT/CALP/DATA/"

Information on the models is being provided in cmip6csvinfo.csv.  This contains information on the models, number of members and whether there's any special order for the member numbers i.e., if they don't simply go from 1 to N.  Read in this info and set up the dates for each period.

In [16]:
cmip6models = pd.read_csv("../cmip6csvinfo.csv")

ybegp=1948 ; monbegp=1 ; yendp=2014 ; monendp=12 # dates for past period
ybegf=2006 ; monbegf=1 ; yendf=2099 ; monendf=12 # dates for future period

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

Set up shapefile for USA

In [17]:
#USA
shpdir = "/project/cas/islas/python/ecpaper2020/shapefiles/USA/"
shpurl = "https://biogeo.ucdavis.edu/data/gadm3.6/shp/gadm36_USA_shp.zip"
with urlopen(shpurl) as ziploc:
    with ZipFile(BytesIO(ziploc.read())) as zfile:
        zfile.extractall(shpdir)
shpfile_USA = shpdir+"gadm36_USA_1.shp"

In [18]:
models = cmip6models["Model"]
nmodels=models.size

# define 1 deg greid
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

# define multi model arrays
prtrend_2006_2099 = xr.DataArray(np.zeros([nmodels, nlat, nlon]), coords=[models, grid_out["lat"], grid_out["lon"]],
                                dims=['Model', 'lat', 'lon'], name='prtrend_2006_2099')
cor_nino_calp_p = xr.DataArray(np.zeros([nmodels]), coords=[models], dims=['Model'], name='cor_nino_calp_p')
cor_nino_calp_f = xr.DataArray(np.zeros([nmodels]), coords=[models], dims=['Model'], name='cor_nino_calp_f')
calptrend_em = xr.DataArray(np.zeros([nmodels]), coords=[models], dims=['Model'], name='calptrend')

# loop over models
for index, modname in models.iteritems():
    
    # initialize boolean for reusing weights
    wgtfile="/project/cas/islas/python/ecpaper2020/DATASORT/CALP/tmp/wgt_cmip6.nc"
    reusewgt=False
    
    # ---the past period----
    nmems=cmip6models.loc[index, "Nmempast"]
    ftype=cmip6models.loc[index, "ftypep"]
    ptype=cmip6models.loc[index, "ptype"]
    
    for imem in range(1, nmems+1, 1): # loop over members
        if (math.isnan(ptype)):
            memstr="r"+str(imem)+"i1p1f"+str(int(ftype))
        else:
            memstr="r"+str(imem)+"i1p"+str(int(ptype))+"f"+str(int(ftype))
        
        # check for a special order
        changeorder = cmip6models.loc[index, "specialorderpast"]
        if (type(changeorder)==str):
            changeordernp = np.array(changeorder.split(","))
            if (math.isnan(ptype)):
                memstr="r"+str(changeordernp[imem-1])+"i1p1f"+str(int(ftype))
            else:
                memstr="r"+str(changeordernp[imem-1])+"i1p"+str(int(ptype))+"f"+str(int(ftype))
            
        print("Processing past for "+modname+" "+memstr+"...")
        histdir=glob(histpath+"pr/"+modname+"/"+memstr+"/*/")
        histdir=histdir[0]
        pr = read.read_sfc(histdir+"/*.nc", datebegp, dateendp)
    
        
        histdir=glob(histpath+"ts/"+modname+"/"+memstr+"/*/")
        histdir=histdir[0]
        ts = read.read_sfc(histdir+"/*.nc", datebegp, dateendp)
        
        # check array length
        if ((pr.time.size != nmonthsp) or (ts.time.size != nmonthsp)):
            print("something's wrong.  nmonthsp="+str(nmonthsp)+" but got "+str(pr.time.size)+"/"+str(ts.time.size)+" for pr/ts")
            sys.exit("failed at past for "+modname+" "+memstr)
        
        
        # convert to mm/day
        pr['pr']=pr['pr']*86400.

        prdjf = cal.season_ts(pr, 'pr', 'DJF')
        tsdjf = cal.season_ts(ts, 'ts', 'DJF')
        
        if (imem == 1):
            prgather_p = xr.DataArray(np.zeros([nmems, prdjf.time.size, prdjf.lat.size, prdjf.lon.size]), 
                                       coords=[np.arange(0,nmems), prdjf['time'], prdjf['lat'], prdjf['lon']], 
                                       dims=['Member','time','lat','lon'], name='pr')
            tsgather_p = xr.DataArray(np.zeros([nmems, tsdjf.time.size, tsdjf.lat.size, tsdjf.lon.size]), 
                                       coords=[np.arange(0,nmems), tsdjf['time'], tsdjf['lat'], tsdjf['lon']], 
                                       dims=['Member','time','lat','lon'], name='pr')
        
        prgather_p[imem-1,:,:,:]=np.array(prdjf)
        tsgather_p[imem-1,:,:,:]=np.array(tsdjf)
        
    # ----- future period ------
    nmems = cmip6models.loc[index, "Nmemfuture"]
    ftype = cmip6models.loc[index, "ftypef"]
    for imem in range(1, nmems+1, 1): # loop over future members
        if (math.isnan(ptype)):
            memstr="r"+str(imem)+"i1p1f"+str(int(ftype))
        else:
            memstr="r"+str(imem)+"i1p"+str(int(ptype))+"f"+str(int(ftype))

        # check for a special order
        changeorder = cmip6models.loc[index, "specialorderfuture"]
        if (type(changeorder) == str):
            changeordernp=np.array(changeorder.split(","))
            if (math.isnan(ptype)):
                memstr="r"+str(changeordernp[imem-1])+"i1p1f"+str(int(ftype))
            else:
                memstr="r"+str(changeordernp[imem-1])+"i1p"+str(int(ptype))+"f"+str(int(ftype))
        
        print("Processing Future for "+modname+", "+memstr+"...")
        histdir=glob(histpath+"pr/"+modname+"/"+memstr+"/*/")
        histdir=histdir[0]
        rcp85dir=glob(rcp85path+"pr/"+modname+"/"+memstr+"/*/")
        rcp85dir=rcp85dir[0]
        prhist = read.read_sfc(histdir+"/*.nc", datebegf, "2014-12")
        prrcp = read.read_sfc(rcp85dir+"/*.nc", "2015-01", dateendf)
        pr = xr.concat([prhist, prrcp], dim="time", join="override")
        
        histdir=glob(histpath+"ts/"+modname+"/"+memstr+"/*/")
        histdir=histdir[0]
        rcp85dir=glob(rcp85path+"ts/"+modname+"/"+memstr+"/*/")
        rcp85dir=rcp85dir[0]
        tshist = read.read_sfc(histdir+"/*.nc", datebegf, "2014-12")
        tsrcp = read.read_sfc(rcp85dir+"/*.nc", "2015-01", dateendf)
        ts = xr.concat([tshist, tsrcp], dim="time", join="override")
        
        # check array sizes
        if ((pr.time.size != nmonthsf) or (ts.time.size != nmonthsf)):
            print("Something's wrong. nmonthsf="+str(nmonthsf)+" but got "+str(ts.time.size)+"/"+str(ts.time.size)+" for pr/ts")
            sys.exit("failed at future for "+modname+" "+memstr)
        
        # convert to mm/day
        pr['pr'] = pr['pr']*86400.
        
        prdjf = cal.season_ts(pr, 'pr', 'DJF')
        tsdjf = cal.season_ts(ts, 'ts', 'DJF')
        
        if (imem == 1):
            prgather_f = xr.DataArray(np.zeros([nmems, prdjf.time.size, prdjf.lat.size, prdjf.lon.size]), 
                                       coords=[np.arange(0,nmems), prdjf['time'], prdjf['lat'], prdjf['lon']], 
                                       dims=['Member','time','lat','lon'], name='pr')
            tsgather_f = xr.DataArray(np.zeros([nmems, tsdjf.time.size, tsdjf.lat.size, tsdjf.lon.size]), 
                                       coords=[np.arange(0,nmems), tsdjf['time'], tsdjf['lat'], tsdjf['lon']], 
                                       dims=['Member','time','lat','lon'], name='pr')
        
        prgather_f[imem-1,:,:,:]=np.array(prdjf)
        tsgather_f[imem-1,:,:,:]=np.array(tsdjf)
        
        
    # interpolate to 1 degree grid and calculate area averages
    regridder = xe.Regridder(prgather_p, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
    prgather_p_interp = regridder(prgather_p)
    reusewgt=True
    regridder = xe.Regridder(prgather_f, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
    prgather_f_interp = regridder(prgather_f)
    regridder = xe.Regridder(tsgather_p, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
    tsgather_p_interp = regridder(tsgather_p)
    regridder = xe.Regridder(tsgather_f, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
    tsgather_f_interp = regridder(tsgather_f)
    
    prgather_f_em = prgather_f_interp.mean("Member")
    
    prgather_p_detrend = xr.DataArray(signal.detrend(prgather_p_interp, axis=1), coords=prgather_p_interp.coords)
    prgather_f_detrend = xr.DataArray(signal.detrend(prgather_f_interp, axis=1), coords=prgather_f_interp.coords)
    tsgather_p_detrend = xr.DataArray(signal.detrend(tsgather_p_interp, axis=1), coords=tsgather_p_interp.coords)
    tsgather_f_detrend = xr.DataArray(signal.detrend(tsgather_f_interp, axis=1), coords=tsgather_f_interp.coords)   
    
    nino34_p = avg.cosweightlonlat(tsgather_p_detrend, 190, 240, -5, 5)
    nino34_f = avg.cosweightlonlat(tsgather_f_detrend, 190, 240, -5, 5)
    
    if (index == 0) and (imem == 1): # only need to make the mask for 1 deg  once
        mask = shp.maskgen(shpfile_USA, prgather_p_detrend, ['California'])
    
    prgather_p_mskd = xr.DataArray(np.array(prgather_p_detrend)*np.array(mask), coords=prgather_p_detrend.coords)
    prgather_f_mskd = xr.DataArray(np.array(prgather_f_detrend)*np.array(mask), coords=prgather_f_detrend.coords)
    prcal_p=avg.cosweightlonlat(prgather_p_mskd, 0, 360, -90, 90)
    prcal_p = prcal_p.rename('pr')
    prcal_f=avg.cosweightlonlat(prgather_f_mskd, 0, 360, -90, 90)
    prcal_f = prcal_f.rename('pr')
    
    
    cor_nino_calp_p[index] = xr.corr(prcal_p, nino34_p)
    cor_nino_calp_f[index] = xr.corr(prcal_f, nino34_f)

    # convert time axis to year for detrending
    year = prgather_f_em['time'].dt.year
    prgather_f_em['time'] = year
    
    a, b = linfit.linfit_lonlat(prgather_f_em, "time")
    prtrend_2006_2099[index,:,:]=b*100.
    
prtrend_mskd = xr.DataArray(np.array(prtrend_2006_2099)*np.array(mask), coords=prtrend_2006_2099.coords)
prtrend_cal = avg.cosweightlonlat(prtrend_mskd, 0, 360, -90, 90) 
prtrend_cal = prtrend_cal.rename('prtrend_cal')

Processing past for ACCESS-CM2 r1i1p1f1...
Processing past for ACCESS-CM2 r2i1p1f1...
Processing Future for ACCESS-CM2, r1i1p1f1...
masking California
Processing past for ACCESS-ESM1-5 r1i1p1f1...
Processing past for ACCESS-ESM1-5 r2i1p1f1...
Processing past for ACCESS-ESM1-5 r3i1p1f1...
Processing Future for ACCESS-ESM1-5, r1i1p1f1...
Processing Future for ACCESS-ESM1-5, r2i1p1f1...
Processing Future for ACCESS-ESM1-5, r3i1p1f1...
Processing past for AWI-CM-1-1-MR r1i1p1f1...
Processing past for AWI-CM-1-1-MR r2i1p1f1...
Processing past for AWI-CM-1-1-MR r3i1p1f1...
Processing past for AWI-CM-1-1-MR r4i1p1f1...
Processing past for AWI-CM-1-1-MR r5i1p1f1...
Processing Future for AWI-CM-1-1-MR, r1i1p1f1...
Processing past for BCC-CSM2-MR r1i1p1f1...
Processing past for BCC-CSM2-MR r2i1p1f1...
Processing past for BCC-CSM2-MR r3i1p1f1...
Processing Future for BCC-CSM2-MR, r1i1p1f1...
Processing past for CAMS-CSM1-0 r1i1p1f1...
Processing Future for CAMS-CSM1-0, r1i1p1f1...
Processing Futu

In [20]:
calpvalues = xr.merge([prtrend_cal, cor_nino_calp_p, cor_nino_calp_f])
calpvalues.to_netcdf(path=pathout+"CMIP6calpvalues.nc")

prtrend_2006_2099.to_netcdf(path=pathout+"CMIP6calpmaptrends.nc")